In [131]:
import pandas as pd
from Levenshtein import distance
# import jellyfish
# import textdistance
pd.set_option('display.max_rows', None)

year = 2026

transfers_df = pd.read_csv(f"DataGovCSVs/TIF_Transfers_{year}.csv")
# transfers_df["TIF District 2 CAPS"] = transfers_df["TIF District 2"].str.upper()
# distance = jellyfish.jaro_winkler_similarity
# distance = textdistance.levenshtein.normalized_distance

In [132]:
SBIF = transfers_df[transfers_df["TIF2"] == "SBIF"]
SBIF

,TIF Number,TIF1,TIF2,Direction,Amount,Project


In [133]:
transfers_df = transfers_df[transfers_df["TIF2"] != "SBIF"]

In [134]:
vals1 = set(transfers_df["TIF1"].unique())
vals2 = set(transfers_df["TIF2"].unique())

print(len(vals1))
print(len(vals2))

10
10


In [135]:
def get_match(str1:str, values:list[str]):
    closest_distance = float('inf')
    closest_value = None
    for value2 in values:
        if value2 == "SBIF":
            continue
        if ((False not in [word.lower() in value2.lower() for word in str1.split(' ')]) or 
            (False not in [word.lower() in str1.lower() for word in value2.split(' ')])):
            return value2
        dist = distance(str1.lower(), value2.lower())
        if dist < closest_distance:
            closest_distance = dist
            closest_value = value2
    return closest_value

def find_closest_match(df:pd.DataFrame, column1:str, column2:str):
    # Extract the columns
    col1 = list(df[column1].unique())
    col2 = list(df[column2].unique())
    
    limbo = []

    # Find closest match for each element in col1
    closest_mapping = {}
    closest_mapping_swapped = {None: []}

    for TIF in col2:
        closest_value = get_match(TIF, col1)
        
        if (closest_value in closest_mapping_swapped):
            limbo.append((TIF, []))
        
        while (limbo):
            new_key = limbo[0][0]
            tries = limbo[0][1]
            value = get_match(TIF, [x for x in col2 if x not in tries])
            set_key = closest_mapping_swapped[value] if value in closest_mapping_swapped else None

            if not value:
                # print('new_key: ', new_key)
                # print('tries: ', tries)
                # print('value: ', value)
                # print('set_key: ', set_key)
                # print('closest_mapping: ', closest_mapping)
                # print('closest_mapping_swapped: ', closest_mapping_swapped)
                closest_mapping[new_key] = value
                closest_mapping_swapped[value].append(new_key)
                limbo.pop()
            elif (set_key == None):
                closest_mapping[new_key] = value
                closest_mapping_swapped[value] = new_key
                limbo.pop()
            elif (distance(set_key, value) > distance(new_key, value)):
                del closest_mapping[set_key]
                del closest_mapping_swapped[value]
                closest_mapping[new_key] = value
                closest_mapping_swapped[value] = new_key
                limbo.pop()
                limbo.append((set_key, []))
            else:
                limbo[0][1].append(value)

        closest_mapping[TIF] = closest_value
        closest_mapping_swapped[closest_value] = TIF

    # Create a new DataFrame with replaced values
    new_df = pd.DataFrame([(k,v) for k,v in closest_mapping.items()], columns = ([column2, column1]))

    return new_df, closest_mapping_swapped[None]
    # return new_df, None

In [136]:
matches, nones = find_closest_match(transfers_df, "TIF1", "TIF2")    

In [137]:
matches

,TIF2,TIF1
0,119th/I-57,119th/I-57
1,107th/Halsted,107th/Halsted
2,Roosevelt Clark,Roosevelt Clark
3,Northwest Ind. Corr.,Northwest Industrial Corridor
4,Ogden/Pulaski,Ogden/Pulaski
5,Roseland/Michigan,Roseland/Michigan
6,Madison/Austin,Madison/Austin Corridor
7,Midwest,Midwest
8,Canal/Congress,Canal/Congress
9,North Pullman,North Pullman


In [138]:
for tif in transfers_df["TIF2"]:
    if not (tif in list(matches["TIF2"])):
        print(tif)

In [139]:
for tif in transfers_df["TIF1"]:
    if not (tif in list(matches["TIF1"])):
        print(tif)

In [140]:
conversions = {row["TIF2"]:row["TIF1"] for index, row in matches.iterrows()}
if year == 2021:
    conversions["Stony Island/Burnside"] = "Stony Island Avenue Commercial and Burnside Industrial Corridors"
elif year == 2022:
    conversions["Woodlawn"] = "Woodlawn"
    conversions["Stony Island/Burnside"] = "Stony Island Avenue Commercial and Burnside Industrial Corridors"
elif year == 2023:
    conversions["Greater SW Ind. Corr. East"] = "Greater Southwest Industrial Corridor (East)"
    conversions["Greater SW Ind. Corr. West"] = "Greater Southwest Industrial Corridor (West)"
    conversions["Stony Island/Burnside"] = "Stony Island Avenue Commercial and Burnside Industrial Corridors"
    conversions["Woodlawn"] = "Woodlawn"
    conversions["119th/Halsted"] = "119th and Halsted"
elif year == 2024:
    conversions["Greater SW Ind. Corr. East"] = "Greater Southwest Industrial Corridor (East)"
    conversions["119th/Halsted"] = "119th and Halsted"
    conversions["79th/Southwest Hwy"] = "79th Street/Southwest Highway"


In [141]:
conversions

{'119th/I-57': '119th/I-57',
 '107th/Halsted': '107th/Halsted',
 'Roosevelt Clark': 'Roosevelt Clark',
 'Northwest Ind. Corr.': 'Northwest Industrial Corridor',
 'Ogden/Pulaski': 'Ogden/Pulaski',
 'Roseland/Michigan': 'Roseland/Michigan',
 'Madison/Austin': 'Madison/Austin Corridor',
 'Midwest': 'Midwest',
 'Canal/Congress': 'Canal/Congress',
 'North Pullman': 'North Pullman'}

In [142]:
transfers_df["TIF2"] = transfers_df["TIF2"].replace(conversions)
transfers_df

,TIF Number,TIF1,TIF2,Direction,Amount,Project
0,T-176,107th/Halsted,119th/I-57,From,1000000,Invest South West Program
1,T-125,119th/I-57,107th/Halsted,To,-1000000,Invest South West Program
2,T-063,Canal/Congress,Roosevelt Clark,To,-24285800,Infra-Structure - Roosevelt Clark TIF
3,T-075,Madison/Austin Corridor,Northwest Industrial Corridor,From,1500000,Invest South West Program
4,T-095,Midwest,Ogden/Pulaski,To,-6408000,Invest South West Program
5,T-166,North Pullman,Roseland/Michigan,From,145000,RDA - Whole Foods Distribution
6,T-064,Northwest Industrial Corridor,Madison/Austin Corridor,To,-1500000,Invest South West Program
7,T-161,Ogden/Pulaski,Midwest,From,6408000,Invest South West Program
8,T-184,Roosevelt Clark,Canal/Congress,From,24285800,Infra-Structure - Roosevelt Clark TIF
9,T-113,Roseland/Michigan,North Pullman,To,-145000,RDA - Whole Foods Distribution


In [143]:
transfers_df.to_csv(f"DataGovCSVs/TIF_Transfers_{year}_modified.csv")